In [1]:
import get_combine_data
import altair as alt

/Users/caseydurfee/.virtualenvs/py3-data/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.0.0)/charset_normalizer (2.0.11) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
all_data = get_combine_data.get_data(position_key='general_position')

draft_pos_corr = get_combine_data.most_corr_with_draft_pos(all_data)

/Users/caseydurfee/msds/data_viz/get_combine_data.py:198: FutureWarning: The default value of numeric_only in DataFrameGroupBy.corr is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  abs_corrs = all_data.groupby(by="general_position").corr()["DraftNumber"].abs()


In [3]:
chart = alt.Chart(draft_pos_corr)

scale = alt.Scale(scheme='redyellowblue', reverse=True)

chart = chart.mark_rect().encode(
    alt.X('Rank:O', axis=alt.Axis(orient='top', labelAngle=0)),
    y='Position:O',
    color=alt.Color('Importance:Q', scale=scale),
    tooltip=alt.Tooltip('Importance:O', format=".3")
)

text = chart.mark_text(baseline='middle', fontSize=14).encode(
    alt.Text('Event:O'),
    color=alt.value("black")
)

#chart=chart.add_selection(point)

chart = chart.properties(
    width=600, height=600
).properties(
    title="Influence of combine results on draft position (r^2)"
)


First, I am going to show a heatmap of which exercises are the most/least important as far as draft order.

The goal was to make it easy to tell which were most important at a glance.

The correlation is with the relative performance of the athlete compared to others at their position, not all athletes.

The hover shows the exact r^2 value, while the color shows it with a hot/cold spectrum.

In [4]:
chart + text

/Users/caseydurfee/.virtualenvs/py3-data/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

Next, I wanted to show the distribution of exercise results for each position.

In [15]:
step = 40
overlap = 0

DEFAULT_POSITION = "RB"

alt.data_transformers.disable_max_rows()

eav = get_combine_data.quantiles_as_eav(all_data, position_key='general_position')

len(eav.dropna())

DROPDOWN_POSITIONS = list(eav.position.unique())

input_dropdown = alt.binding_select(options=DROPDOWN_POSITIONS, name='position')

position_selection = alt.selection_single(fields=['position'], bind=input_dropdown, init={"position": "Running Back"})

c = alt.Chart(eav, height=step
).add_selection(
    position_selection
).transform_filter(
    position_selection
).transform_joinaggregate(
    mean_value='mean(result)', groupby=['event']
).transform_bin(
    ['bin_min', 'bin_max'], "result", bin=alt.Bin(maxbins=100)
).transform_aggregate(
    value='count()', groupby=['event', 'mean_value', 'bin_min', 'bin_max']
).transform_impute(
    impute='value', groupby=['event', 'mean_value'], key='bin_min', value=0
).mark_area(

).encode(
    alt.X('bin_min:Q', scale=alt.Scale(domain=[0,100]), title=None),
    alt.Y(
        'value:Q',
        scale=alt.Scale(range=[step, -step * overlap]),
        axis=None
    ),
    alt.Fill(
        'mean_value:Q',
        #legend=None,
        scale=alt.Scale(domain=[20, 80], scheme='redyellowblue', reverse=True),
        title='mean percentile'
    ),
    alt.Row('event:N', title=None, header=alt.Header(labelAngle=0, labelAlign='left')),
    tooltip=alt.Tooltip(['mean_value:Q'], format=".2", title='mean percentile')
)
c.properties(
    title='Percentile performance vs. all drafted players',
    bounds='flush'
).configure_view(
    stroke=None
).configure_title(
    anchor='middle',
    align='center'
)

/Users/caseydurfee/.virtualenvs/py3-data/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)